In [72]:
import json
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
from scipy.special import stirling2
import math
from copy import copy

In [73]:
status_codes = {
    1: 'LOADED',
    2: 'OPTIMAL',
    3: 'INFEASIBLE',
    4: 'INF_OR_UNBD',
    5: 'UNBOUNDED',
    6: 'CUTOFF',
    7: 'ITERATION_LIMIT',
    8: 'NODE_LIMIT',
    9: 'TIME_LIMIT',
    10: 'SOLUTION_LIMIT',
    11: 'INTERRUPTED',
    12: 'NUMERIC',
    13: 'SUBOPTIMAL',
    14: 'INPROGRESS',
    15: 'USER_OBJ_LIMIT'
}

In [74]:
rng = np.random.default_rng(8)

# SDP moment equations for general model

Take as arguments
- number of reactions R and a_r, v_r for each reaction
- number of species S and set of unobserved species indices U
- d
- additional constraints e.g. telegraph moments, independence, etc

Should allow unified construction of all 4 models done so far, as well as choices such as 2 species independent models, etc

- feasibility test function needs to take all data about reactions and species
- BUT reaction rates defined via sympy variables so use eval on strings

- want to link to existing package
- BUT even dataset class has lots of unneccessary information (truncations, probabilities, etc)
- consider creating a new package

- try model free moments with SDP moment validity constraints / factorization of higher order moments than just cross = mean * mean

In [75]:
str_poly = "xs[0] * xs[1]"

In [76]:
xs = sp.symbols([f'x{i}' for i in range(2)])
poly = eval(str_poly)
#poly = sp.Poly(poly)
poly.coeff

<bound method Expr.coeff of x0*x1>

In [77]:
sp.parse_expr(str_poly, {'xs': xs})

x0*x1

## Helper functions

In [78]:
def compute_order(alpha):
    order = 0
    for alpha_i in alpha:
        order += alpha_i
    return order

In [79]:
def compute_Nd(S, d):
    '''Number of moments of order <= d (S species)'''
    Nd = math.factorial(S + d) // (math.factorial(d) * math.factorial(S))
    return Nd

In [80]:
def compute_powers(S, d):
    '''Compute the Nd powers of order <= d (S species)'''

    # all powers
    powers = [[0 for s in range(S)]]

    # powers of order d = 0
    powers_prev = [[0 for s in range(S)]]

    # for order d = 1, ..., d
    for order in range(1, d + 1):

        # store powers of order d
        powers_current = []

        # for each power of order d - 1
        for alpha in powers_prev:

            # for each index
            for i in range(S):

                # add 1 to power at index
                alpha_new = copy(alpha)
                alpha_new[i] += 1

                # store (avoid repeats)
                if alpha_new not in powers_current:
                    powers_current.append(alpha_new)

        # update d - 1 powers
        powers += powers_current

        # update overall powers
        powers_prev = powers_current

    return powers

In [81]:
def add_powers(*powers, S):
    plus = [0 for i in range(S)]
    for i in range(S):
        for power in powers:
            plus[i] += power[i]
    return plus

## Compute A

In [82]:
def compute_A(alpha, reactions, vrs, db, R, S, d):
    '''
    Moment equation coefficient matrix
    NOTE: must have order of alpha <= d

    Args:
        alpha: moment order for equation (d/dt mu^alpha = 0)
        reactions: list of strings detailing a_r(x) for each reaction r
        vrs: list of lists detailing v_r for each reaction r
        db: largest order a_r(x)
        R: number of reactions
        S: number of species
        d: maximum moment order used (must be >= order(alpha) + db - 1)

    Returns:
        A: (R, Nd) matrix of coefficients
    '''

    if compute_order(alpha) > d - db + 1:
        raise NotImplementedError(f"Maximum moment order {d} too small for moment equation of alpha = {alpha}: involves moments of higher order.")


    xs = sp.symbols([f'x{i}' for i in range(S)])

    # reaction propensity polynomials
    # props = [eval(str_ar) for str_ar in reactions]
    props = [sp.parse_expr(str_ar, {'xs': xs}) for str_ar in reactions]

    # number of moments of order <= d
    Nd = compute_Nd(S, d)

    # get powers of order <= d
    powers = compute_powers(S, d)

    # setup matrix
    A = np.zeros((R, Nd))

    for r, prop in enumerate(props):

        # expand b(x) * ((x + v_r)**alpha - x**alpha)
        term_1 = 1
        term_2 = 1
        for i in range(S):
            term_1 = term_1 * (xs[i] + vrs[r][i])**alpha[i]
            term_2 = term_2 * xs[i]**alpha[i]
        poly = sp.Poly(prop * (term_1 - term_2), xs)

        # loop over terms
        for xs_power, coeff in zip(poly.monoms(), poly.coeffs()):

            # get matrix index
            col = powers.index(list(xs_power))

            # store
            A[r, col] = coeff

    return A

In [83]:
compute_A(
    alpha = [1],
    reactions = [
        "1",
        "xs[0]"
    ],
    vrs = [
        [1],
        [-1]
    ],
    db = 1,
    R = 2,
    S = 1,
    d = 3
)

array([[ 1.,  0.,  0.,  0.],
       [ 0., -1.,  0.,  0.]])

### NOTE: need to use xs[0] even if S = 1, using xs will not work

## Compute B

In [84]:
def compute_B(beta, S, U, d):
    '''
    Capture efficiency moment scaling matrix

    Args:
        beta: per cell capture efficiency sample
        S: number of species
        U: unobserved species indices
        d: maximum moment order used

    Returns:
        B: (Nd, Nd) matrix of coefficients
    '''

    # number of moments of order <= d
    Nd = compute_Nd(S, d)

    # compute powers of order <= d
    powers = compute_powers(S, d)

    # compute beta moments of order <= d
    y_beta = np.zeros(d + 1)
    for l in range(d + 1):
        y_beta[l] = np.mean(beta**l)

    def falling_factorial(n, k):
        val = 1
        for i in range(k):
            val *= (n - i)
        return val

    def binomial_moment(n, p, l):
        val = 0
        for k in range(l + 1):
            val += falling_factorial(n, k) * stirling2(l, k) * p**k
        return val

    # setup matrix
    B = np.zeros((Nd, Nd))

    p = sp.Symbol('p')
    xs = sp.symbols([f'x{i}' for i in range(S)])

    # for each moment power
    for row, alpha in enumerate(powers):

        # setup polynomail
        poly_alpha = 1

        # for each species
        for i in range(S):

            # unobserved: no capture efficiency
            if i in U:
                moment = xs[i]**alpha[i]

            # observed: compute moment expression for E[Xi^alphai] in xi
            else:
                moment = binomial_moment(xs[i], p, alpha[i])
            
            poly = sp.Poly(moment, p, xs[i])

            # multiply
            poly_alpha = poly_alpha * poly

        # loop over terms
        for (beta_power, *xs_power), coeff in zip(poly_alpha.monoms(), poly_alpha.coeffs()):

            # get matrix index
            col = powers.index(xs_power)

            B[row, col] += coeff * y_beta[beta_power]

    return B

In [85]:
compute_B(
    beta = rng.beta(1, 2, size=1000),
    S = 1,
    U = [],
    d = 3
)

array([[1.        , 0.        , 0.        , 0.        ],
       [0.        , 0.33578118, 0.        , 0.        ],
       [0.        , 0.16722535, 0.16855583, 0.        ],
       [0.        , 0.03355213, 0.20050983, 0.10171922]])

## Compute M

In [86]:
def construct_M_s(y, s, S, d):
    '''Moment matrix variable constructor (s).'''
    if s == 0:
        D = math.floor(d / 2)
    else:
        D = math.floor((d - 1) / 2)
    powers_D = compute_powers(S, D)
    powers_d = compute_powers(S, d)
    ND = compute_Nd(S, D)
    M_s = [[0 for j in range(ND)] for i in range(ND)]
    e_s = [1 if i == (s - 1) else 0 for i in range(S)]
    for alpha_index, alpha in enumerate(powers_D):
        for beta_index, beta in enumerate(powers_D):
            plus = add_powers(alpha, beta, e_s, S=S)
            plus_index = powers_d.index(plus)
            M_s[alpha_index][beta_index] = y[plus_index].item()
    M_s = gp.MVar.fromlist(M_s)
    return M_s

## General constraint constructors

## Base model

In [96]:
def base_model(model, OB_bounds, beta, reactions, vrs, db, R, S, U, d, fixed=[], time_limit=300,
               moment_bounds=True, moment_matrices=True, moment_equations=True, factorization=False, telegraph=False):
    '''
    Construct 'base model' with semidefinite constraints removed to give NLP

    Args:
        model: empty gurobi model object
        OB_bounds: confidence intervals on observed moments up to order d (at least)
        beta: capture efficiency vector
        reactions: list of strings detailing a_r(x) for each reaction r
        vrs: list of lists detailing v_r for each reaction r
        db: largest order a_r(x)
        R: number of reactions
        S: number of species
        U: indices of unobserved species
        d: maximum moment order used
        fixed: list of pairs of (reaction index r, value to fix k_r to)
        time_limit: optimization time limit

        constraint options

        moment_bounds: CI bounds on moments
        moment_matrices: 
        moment_equations
        factorization
        telegraph_moments


    Returns:
        model: gurobi model object with NLP constraints (all but semidefinite)
        variables: dict for model variable reference
    '''

    # model settings
    model.Params.TimeLimit = time_limit
    K = 100

    # helpful values
    Nd = compute_Nd(S, d)

    # variables
    y = model.addMVar(shape=Nd, vtype=GRB.CONTINUOUS, name="y", lb=0)
    k = model.addMVar(shape=R, vtype=GRB.CONTINUOUS, name="k", lb=0, ub=K)

    # variable dict
    variables = {
        'y': y,
        'k': k
    }

    if moment_matrices:

        # moment matrices
        for s in range(S + 1):
            M_s = construct_M_s(y, s, S, d)
            variables[f'M_{s}'] = M_s
    
    # constraints

    if moment_bounds:
    
        # confidence interval bounds on OB moments (up to order d)
        y_lb = OB_bounds[0, :Nd]
        y_ub = OB_bounds[1, :Nd]

        # B scaling matrix
        B = compute_B(beta, S, U, d)

        # moment bounds
        model.addConstr(B @ y <= y_ub, name="y_UB")
        model.addConstr(B @ y >= y_lb, name="y_LB")

    if moment_equations:

        # moment equations (order(alpha) <= d - db + 1)
        moment_powers = compute_powers(S, d - db + 1)
        for alpha in moment_powers:
            A_alpha_d = compute_A(alpha, reactions, vrs, db, R, S, d)
            model.addConstr(k.T @ A_alpha_d @ y == 0, name=f"ME_{alpha}_{d}")

    if factorization:

        # factorization bounds
        powers = compute_powers(S, d)
        for i, alpha in enumerate(powers):

            # E[X1^a1 X2^a2] = E[X1^a1] E[X2^a2]
            if (alpha[0] > 0) and (alpha[1] > 0):
                j = powers.index([alpha[0], 0])
                l = powers.index([0, alpha[1]])
                model.addConstr(y[i] == y[j] * y[l], name=f"Moment_factorization_{alpha[0]}_({alpha[1]})")

    if telegraph:

        # telegraph moment equality (as Gi in {0, 1}, E[Gi^n] = E[Gi] for n > 0, same with cross moments)
        powers = compute_powers(S, d)
        for i, alpha in enumerate(powers):

            # G1, G2 powers > 0: equal to powers of 1
            if (alpha[2] > 0) and (alpha[3] > 0):
                j = powers.index([alpha[0], alpha[1], 1, 1])
                model.addConstr(y[i] == y[j], name="Telegraph_moment_equality_G1_G2")
            
            # G1 power > 0: equal to power of 1
            elif (alpha[2] > 0):
                j = powers.index([alpha[0], alpha[1], 1, alpha[3]])
                model.addConstr(y[i] == y[j], name="Telegraph_moment_equality_G1")

            # G2 power > 0: equal to power of 1
            elif (alpha[3] > 0):
                j = powers.index([alpha[0], alpha[1], alpha[2], 1])
                model.addConstr(y[i] == y[j], name="Telegraph_moment_equality_G2")

    # fixed moment
    model.addConstr(y[0] == 1, name="y0_base")

    # fixed parameters
    for r, val in fixed:
        model.addConstr(k[r] == val, name=f"k{r}_fixed")

    return model, variables

## General optimization

In [88]:
def optimize(model):
    '''Optimize model with no objective, return status.'''

    # optimize
    model.setObjective(0, GRB.MINIMIZE)
    model.optimize()
    status = status_codes[model.status]

    return model, status

In [89]:
def semidefinite_cut(model, variables, S, print_evals=False, eval_eps=10**-6, printing=False):
    '''
    Check semidefinite feasibility of NLP feasible point
    Feasible: stop
    Infeasible: add cutting plane (ALL negative eigenvalues)

    Args:
        model: optimized NLP model
        variables: model variable reference dict
        print_evals: option to display moment matrix eigenvalues (semidefinite condition)

    Returns:
        model: model with any cutting planes added
        bool: semidefinite feasibility status
    '''

    # data list
    data = []

    # moment matrix values
    for s in range(S + 1):
        data.append(
            {f'M_val': variables[f'M_{s}'].X}
        )

    # eigen information
    for s in range(S + 1):
        evals_s, evecs_s = np.linalg.eigh(data[s]['M_val'])
        data[s]['evals'] = evals_s
        data[s]['evecs'] = evecs_s

    if print_evals:
        print("Moment matices eigenvalues:")
        for s in range(S + 1):
            print(data[s]['evals'])

    # check if all positive eigenvalues
    positive = True
    for s in range(S + 1):
        if not (data[s]['evals'] >= -eval_eps).all():
            positive = False
            break

    # positive eigenvalues
    if positive:

        if printing: print("SDP feasible\n")
    
        return model, True

    # negative eigenvalue
    else:

        if printing: print("SDP infeasible\n")

        # for each matrix
        for s in range(S + 1):

            # for each M_s eigenvalue
            for i, lam in enumerate(data[s]['evals']):

                # if negative (sufficiently)
                if lam < -eval_eps:

                    # get evector
                    v = data[s]['evecs'][:, i]

                    # add cutting plane
                    model.addConstr(v.T @ variables[f'M_{s}'] @ v >= 0, name=f"Cut_{s}")
                
                    if printing: print(f"M_{s} cut added")

        if printing: print("")

    return model, False

In [98]:
def feasibility_test(OB_bounds, beta, reactions, vrs, db, R, S, U, d, fixed=[], time_limit=300,
               moment_bounds=True, moment_matrices=True, moment_equations=True, factorization=False, telegraph=False,
               print_evals=False, printing=False, eval_eps=10**-6):
    '''
    Full feasibility test of birth death model via following algorithm

    Optimize NLP
    Infeasible: stop
    Feasible: check SDP feasibility
        Feasible: stop
        Infeasible: add cutting plane and return to NLP step

    Args:
        OB_bounds: confidence intervals on observed moments up to order d (at least)
        beta: capture efficiency vector
        reactions: list of strings detailing a_r(x) for each reaction r
        vrs: list of lists detailing v_r for each reaction r
        db: largest order a_r(x)
        R: number of reactions
        S: number of species
        U: indices of unobserved species
        d: maximum moment order used
        fixed: list of pairs of (reaction index r, value to fix k_r to)
        time_limit: optimization time limit

        constraint options

        moment_bounds: CI bounds on moments
        moment_matrices: 
        moment_equations
        factorization
        telegraph_moments

        optimization options

        print_evals: toggle printing of moment matrix eigenvalues
        printing: toggle printing of feasibility status
        eval_eps: threshold of allowed negative eigenvalues for semidefinite
        
    Returns:
        None
    '''

    # silent
    options = {'OutputFlag': 0}

    # environment context
    with gp.Env(params=options) as env:

        # model context
        with gp.Model('test-SDP', env=env) as model:

            # construct base model (no semidefinite constraints)
            model, variables = base_model(model, OB_bounds, beta, reactions, vrs, db, R, S, U, d, fixed, time_limit,
                                          moment_bounds, moment_matrices, moment_equations, factorization, telegraph)

            # check feasibility
            model, status = optimize(model)

            # no semidefinite constraints: just return status
            if not moment_matrices:

                if printing: print(status)

                return status

            # while feasible
            while status == "OPTIMAL":

                if printing: print("NLP feasible")

                # check semidefinite feasibility
                model, semidefinite_feas = semidefinite_cut(model, variables, S, print_evals=print_evals, eval_eps=eval_eps, printing=printing)

                # semidefinite feasible
                if semidefinite_feas:
                    break

                # semidefinite infeasible
                else:

                    # check feasibility with added cut
                    model, status = optimize(model)

            # if infeasible
            if status == "INFEASIBLE":
                
                if printing: print("SDP infeasible")

                #model.computeIIS()
                #model.write('test.ilp')

            return status

## Bootstrap

In [91]:
def bootstrap(sample, S, U, d, N=1000):
    '''
    Estimate bootstrap intervals of sample moments up to order d

    Args:
        sample: integer sample of length n
        S: number of species
        U: indices of unobserved species
        d: maximum moment order to estimate
        N: number of bootstrap samples

    Returns:
        y_bounds: (2, Nd) array of moment confidence intervals
    '''

    # helpful values
    powers = compute_powers(S, d)
    Nd = compute_Nd(S, d)

    # get sample size
    n = len(sample)

    # separate sample pairs
    x1_sample = [x[0] for x in sample]
    x2_sample = [x[1] for x in sample]

    # convert sample to n x 2 array
    sample = np.array([x1_sample, x2_sample]).T

    # bootstrap to N x n x 2 array
    boot = rng.choice(sample, size=(N, n))

    # split into 2 N x n arrays
    x1_boot = boot[:, :, 0]
    x2_boot = boot[:, :, 1]

    # estimate
    y_bounds = np.zeros((2, Nd))
    for i, alpha in enumerate(powers):

        # check if unobserved species present in moment
        unobserved_moment = False
        for j, alpha_j in enumerate(alpha):
            if (j in U) and (alpha_j > 0):
                unobserved_moment = True

        # unobserved: [0, inf] bounds
        if unobserved_moment:
            y_bounds[:, i] = np.array([0, np.inf])

        # otherwise: estimate
        else:

            # raise boot to powers
            x1_boot_alpha = x1_boot**alpha[0]
            x2_boot_alpha = x2_boot**alpha[1]

            # multiply (N x n)
            boot_alpha = x1_boot_alpha * x2_boot_alpha

            # mean over sample axis (N x 1)
            moment_estimates = np.mean(boot_alpha, axis=1)

            # quantile over boot axis (2 x 1)
            moment_interval = np.quantile(moment_estimates, [0.025, 0.975])

            # store
            y_bounds[:, i] = moment_interval

    return y_bounds

In [92]:
def downsample_data(sample, b):

    n = len(sample)

    # capture efficiency
    if b == 0:
        beta = np.ones(n)
    else:
        beta = rng.beta(1, b, size=1000)

    # split
    x1_sample = [x[0] for x in sample]
    x2_sample = [x[1] for x in sample]

    # downsample
    x1_downsample = rng.binomial(x1_sample, beta).tolist()
    x2_downsample = rng.binomial(x2_sample, beta).tolist()

    # combine
    downsample = list(zip(x1_downsample, x2_downsample))

    return downsample, beta

## Test

In [63]:
from interaction_inference.simulation import gillespie_birth_death

In [64]:
# settings
k1 = 5
k2 = 1
k_reg = 5
b = 1
n = 1000
N = 1000

# sample
params = {
    'k_tx_1': k1,
    'k_tx_2': k1,
    'k_deg_1': k2,
    'k_deg_2': k2,
    'k_reg': k_reg
}
sample = gillespie_birth_death(params, n)

# downsample
downsample, beta = downsample_data(sample, b)

# mean expression level
print(f"Mean expression {np.mean(downsample)}")

Mean expression 0.6655


In [99]:
# settings
reactions = [
    "1",
    "xs[0]",
    "1",
    "xs[1]",
    "xs[0] * xs[1]"
]
vrs = [
    [1, 0],
    [-1, 0],
    [0, 1],
    [0, -1],
    [-1, -1]
]
db = 2
R = 5
S = 2
U = []
d = 3
fixed = [(1, 1)]

# bootstrap
y_bounds = bootstrap(downsample, S, U, d, N)

# test feasibility
status = feasibility_test(
    y_bounds,
    beta,
    reactions,
    vrs,
    db,
    R,
    S,
    U,
    d,
    fixed=fixed,
    time_limit=300,
    moment_bounds=True,
    moment_matrices=True,
    moment_equations=True,
    factorization=False,
    telegraph=False,
    print_evals=False,
    printing=True,
    eval_eps=10**-6
)

NLP feasible
SDP feasible

